# Test Salmon to quantify TEs with simulated data

In [1]:
import pandas as pd
from Bio import SeqIO
from collections import defaultdict
from benchmark.make_txome import make_txome
from benchmark.simulate import run_polyester

ModuleNotFoundError: No module named 'benchmark'

In [2]:
GENOME_FA = "resources/hg38/hg38.fa"
TXOME_GTF = "resources/hg38/gencode.v43.basic.annotation.gtf"
RMSK = "resources/hg38/hg38.fa.out.gz"

## Generate custom sequences

In [3]:
# ~2 min for chr22
make_txome("GENCODE_USR", GENOME_FA, TXOME_GTF, chromosome="chr22")

INFO:root:Retrieving SVA_E consensus sequence from DFAM API at https://dfam.org/api/families
INFO:root:Retrieving SVA_F consensus sequence from DFAM API at https://dfam.org/api/families
INFO:root:Retrieving AluY consensus sequence from DFAM API at https://dfam.org/api/families
INFO:root:Retrieving HERVK consensus sequence from DFAM API at https://dfam.org/api/families
INFO:root:Retrieving L1HS consensus sequence from REPBASE at /iblm/logglun02/mcuoco/projects/salmonTE_testing/resources/hg38/humrep.ref
INFO:root:Using bedtools from /iblm/logglun02/mcuoco/projects/salmonTE_testing/.conda/bin/bedtools
INFO:root:TE fasta written to /iblm/netapp/data4/mcuoco/tmp/tmp3cf0gk6l.fa
INFO:root:Chromosome chr22 fasta written to /iblm/netapp/data4/mcuoco/tmp/tmp2hbe356g.fa
INFO:root:Chromosome chr22 gtf written to /iblm/netapp/data4/mcuoco/tmp/tmpsusy55lq.gtf
INFO:root:Saving spliceu txome to GENCODE_USR


## Simulate reads with polyester

In [4]:
txome_fa = "GENCODE_USR/txome.fa"

# make count matrix to simulate from
# ~20x coverage ----> reads per transcript = transcriptlength/readlength * 20
# here all transcripts will have ~equal FPKM
# read length = 100
counts = defaultdict(list)
for tx in SeqIO.parse(txome_fa, "fasta"):
    counts["tx_id"].append(tx.id)
    for sample in range(0, 2):
        if "ENS" in tx.id:
            counts[i].append(20 * len(tx.seq) // 100)
        elif "L1HS" in tx.id:
            counts[i].append(i * len(tx.seq) // 100)
        else:
            counts[i].append(0)

counts = pd.DataFrame(counts).set_index("tx_id")

In [5]:
# takes ~30-40 min for 2 samples and chr22
run_polyester(txome_fa, counts, outdir="test", n_jobs=8)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:  2.2min remaining:  6.5min
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:  3.6min remaining:  6.1min
[Parallel(n_jobs=8)]: Done   4 out of   8 | elapsed:  7.8min remaining:  7.8min
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:  9.8min remaining:  5.9min
[Parallel(n_jobs=8)]: Done   6 out of   8 | elapsed: 14.3min remaining:  4.8min
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed: 30.4min finished
INFO:root:Merging simulated reads
